In [1]:
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
from FairRanking.datasets.adult import Adult
import random
from FairRanking.datasets.law import Law
from FairRanking.datasets.compas import Compas
from FairRanking.datasets.wiki import Wiki
from FairRanking.models.DirectRanker import DirectRanker
from FairRanking.models.BaseDirectRanker import convert_data_to_tensors
from FairRanking.models.DirectRankerAdv import DirectRankerAdv
from FairRanking.TrainingFunctions import DirectRankerAdvTrain, DirectRankerTrain
from FairRanking.helpers import rND_torch, nDCG_cls_no_model, auc_estimator, group_pairwise_accuracy, auc_estimator2, nDCG_cls

In [2]:
path = '../Results/DirectRankerAdv/Adult/T-Test/results.csv'
#with open(path, 'w') as file:
#    file.write(f'nDCG,AUC,rND,GPA,Schedule\n')

In [3]:
def write_results(model, X, y, s, schedule, path):
    model.eval()
    with torch.no_grad():
        pred = model.predict_proba(X)
        ndcg_list = []
        for _ in range(100):
            ndcg_i = nDCG_cls(pred, y, esti=False, trec=False, at=500)
            ndcg_list.append(ndcg_i)
        ndcg = np.mean(ndcg_list)
        auc = auc_estimator2(pred, y)
        rnd = rND_torch(pred, torch.argmax(s, dim=1))
        gpa = group_pairwise_accuracy(pred, y, s)
        with open(path, 'a') as file:
            file.write(f'{ndcg},{auc},{rnd},{gpa},{str(schedule).replace(",",";")}\n')


In [9]:
#data = Law('Gender','/Users/robert/Desktop/Bachelor/FairRanker/data')
data = Adult('/Users/robert/Desktop/Bachelor/FairRanker/data')
#data = Compas('/Users/robert/Desktop/Bachelor/FairRanker/data')
#data = Wiki()
n_epochs = 1000
hidden_layers = [64, 32, 16]
bias_layers = [64, 32, 16]
main_lr = 0.01
adv_lr = 0.01
schedules = [[2,1], [3,2]]
for schedule in schedules:
        for i in range(30):
            data.got_data = False
            full_data = data.get_data(random_state=random.randint(1, 100))
            model = DirectRankerAdv(num_features=full_data[0][0].shape[1],
                            kernel_initializer=nn.init.normal_,
                            hidden_layers=hidden_layers,
                            bias_layers=bias_layers,
                            num_classes=len(torch.unique(full_data[0][1])),
                            random_seed=random.random()
                    )
            model = DirectRankerAdvTrain.train(model, full_data, n_epochs=n_epochs, schedule=schedule, threshold=0.0, adv_lr=adv_lr, main_lr=main_lr, print_res=False, random_seed=random.random())
            write_results(model, full_data[2][0], full_data[2][1], full_data[2][2], schedule, path)
            print(f'Finished Schedule: {schedule} run {i}')
print('Finished Scheduling')



Finished Schedule: [2, 1] run 0
Finished Schedule: [2, 1] run 1
Finished Schedule: [2, 1] run 2
Finished Schedule: [2, 1] run 3
Finished Schedule: [2, 1] run 4
Finished Schedule: [2, 1] run 5
Finished Schedule: [2, 1] run 6
Finished Schedule: [2, 1] run 7
Finished Schedule: [2, 1] run 8
Finished Schedule: [2, 1] run 9
Finished Schedule: [2, 1] run 10
Finished Schedule: [2, 1] run 11
